# Optimisation of operation
### Table of Contents

* [1. Introduction](#1.introduction)
* [2. First basic problem](#fbp)
    * [2.1. Math and first step with pyomo for solving the problem](#math)
    * [2.2. Variables ](#variables)
    * [2.3. Constraints](#constraints)
* [3. Extensions of this operation problem ](#temporal)
    * [3.1. Linear temporal coupling with ramp constraints ](#ramp)
    * [3.2. Linear spatial coupling with spatial constraints - Problem Op3 Multi-Area -](#spatial)
* [4. Storage operation ](#storage)
    *[4.1. Optimisation of a storage market participation](#storageMarket)
    *[4.2. Simultaneous optimisation of storage and electric system](#storagecoupling)

## 1. Introduction <a class="anchor" id="1.introduction"></a>
This document contains a description of optimisation tools for electric system operation simulation provided here. It also has a sequence of 4 questions (Q2.1 to Q2.4). Answer to questions can be found in file Question_2_TP.py.

The aim of this part consists in creating a model that enables you to handle the consumption of energy with the lowest prices possible (for the means of production).

In [ ]:
import os
os.chdir('..') ## to work at project root  like in any IDE
InputFolder='Data/input/'
import numpy as np
import pandas as pd
import csv

import datetime
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model

from functions.functions_Operation import *

## 2. First basic problem <a class="anchor" id="fbp"></a>
### 2.1. Math and first step with pyomo for solving problem<a class="anchor" id="math"></a>
\begin{align}
&\text{Cost function }& &\min_{x}  \sum_t \sum_i \pi_i x_{it}\;\;\; & & \pi_i \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{it}\leq a_{it} \bar{x_i} & &\bar{x_i} \text{ installed power, }  a_{it} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{it} \geq  C_t  && C_t \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
\end{align}

If you want to solve the preceeding problem you might have to write everything in a suitable matrix form.
This can be very painful in some cases.
With Pyomo you just need to provide the area consumption, the availability factors and the hypothesis
for the different means of production (in the file Gestion-Simple_TECHNOLOGIES.csv).
Pyomo is then charged of building the matrix form,
and you just have to think about problem formulation.

To build a model, Pyomo needs three different kinds of data :
sets, parameters and variables.

Sets are dimensions, here the time and the name of technology plus a mix of these two :
TIMESTAMP, TECHNOLOGIES and TIMESTAMP_TECHNOLOGIES (product set).

Parameters are tables indexed by set whose values are specified by the user.
Here is the list of the parameters : energycost, EnergyNbhourCap, capacity,
availability factor, area consumption.

variable are tables indexed by set whose values are found by the solver :
the energy produced by each mean of production.

First, we can observe one of the parameters, the availability factor, that has an really important role in the production of electricity

Look at input-XXX.ipynb files to see available input data

In [ ]:
Zones="FR" ; year=2013
#### reading areaConsumption availabilityFactor and TechParameters CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
Selected_TECHNOLOGIES={'Thermal', 'OldNuke'} #you can add technologies here
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

Now we run the optimisation script

In [ ]:
model = GetElectricSystemModel_GestionSingleNode(areaConsumption,availabilityFactor,TechParameters)
opt = SolverFactory('mosek')
results=opt.solve(model)


## 2.2 Analysing results : variables and lagrange multipliers <a class="anchor" id="optiofope"></a>

Let's analyse the result of the optimisation first with **variables**

In [ ]:
Variables=getVariables_panda(model)
Variables['energy'].rename(columns={'energy_index': 'TIMESTAMP', 1: 'TECHNOLOGIES'}, inplace=True)

print(Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')) #pour avoir la production en KWh de chaque moyen de prod chaque heure
print(Variables['energyCosts']) #pour avoir le coût de chaque moyen de prod à l'année

In [ ]:
#graphe montrant la proportion de chaque moyen de prod dans la production totale d'électricité
prod=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
prod['Thermal']=prod['Thermal']+prod['OldNuke']
fig2=go.Figure()
fig2.add_trace(
    go.Scatter(x=list(prod.index), y=list(prod.Thermal),name="production Thermal qui complète le Nuke"))
fig2.add_trace(
    go.Scatter(x=list(prod.index), y=list(prod.OldNuke), name="production Nuke"))
fig2.update_layout(
    title_text="Production électrique (en KWh)",xaxis_title="heures de l'année")
fig2.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="en heures",
                     step="hour",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="-"
    )
)
fig2.show()


We have the **Lagrange multipliers** associated to each (active) **constraint** (zero means unactive constraint).
We have three different constraints : the energy constraint, the capacity constraint and the storage constraint (unactive here)

In [ ]:
Constraints= getConstraintsDual_panda(model)
Constraints['CapacityCtr'].rename(columns={'CapacityCtr_index': 'TIMESTAMP', 1: 'TECHNOLOGIES'}, inplace=True)

# Analyse energyCtr
print(Constraints['energyCtr']['energyCtr']*1000000)

In [ ]:
print(round((Constraints['energyCtr']*1000000).energyCtr,2).unique())

In [ ]:
# Analyse CapacityCtr
print(round((Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000),2))

In [ ]:
round((Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000).OldNuke,2).unique()

Q)2.1. Explain all values of Lagrange multipliers associated to these two constraints. Run the programm with the additional HydroReservoir production mean. This adds a constraint, make the same analysis as before with the Lagrange multipliers associated to the three constraints.

## 3. Extensions of this operation problem <a class="anchor" id="temporal"></a>
### 3.1.  Linear temporal coupling with ramp constraints <a class="anchor" id="ramp"></a>
We go back to problem Op1 and add : dependency on area z (country), add a congestion constraint, ramp constraints (with rc=0,005)

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_z \sum_t \sum_i \pi_{iz} x_{itz}\;\;\; & & \pi_{iz} \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{itz}\leq a_{itz} \bar{x_{iz}} & &\bar{x_{iz}} \text{ installed power, }  a_{itz} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{itz} \geq  C_{tz}  && C_{tz} \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
&\text{ramp limit }   & &rc^-_i *x_{it}\leq x_{it}-x_{i(t+1)}\leq rc^+_i *x_{it} && rc^+_i rc^-_i\text{ ramp limit}\\
\end{align}


In [ ]:
Zones="FR"
year=2013

Selected_TECHNOLOGIES={'Thermal', 'OldNuke'} #you'll add 'Solar' after

#### reading CSV files
areaConsumption = pd.read_csv('CSV/input/areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv('CSV/input/availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv('CSV/input/Gestion-RAMP1_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

In [ ]:
model = GetElectricSystemModel_GestionSingleNode(areaConsumption,availabilityFactor,TechParameters)
opt = SolverFactory('mosek')
results=opt.solve(model)
Variables=getVariables_panda(model)
Constraints= getConstraintsDual_panda(model)
Variables['energy'].rename(columns={'energy_index': 'TIMESTAMP', 1: 'TECHNOLOGIES'}, inplace=True)
Constraints['CapacityCtr'].rename(columns={'CapacityCtr_index': 'TIMESTAMP', 1: 'TECHNOLOGIES'}, inplace=True)

Q)2.2. Write a little script to check if the ramp constraint is verified.

In [ ]:
print(Constraints)

In [ ]:
print(round((Constraints['energyCtr']*1000000).energyCtr,2).unique())

In [ ]:
round((Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000).OldNuke,2).unique()

Q)2.3. Again explain the lagrange multipliers. Add renewable production. How does it changes when you add more renewable ?

### 3.2.  Linear spatial coupling with spatial constraints <a class="anchor" id="spatial"></a>


In [ ]:
Zones="FR_DE_GB_ES"
year=2016
Selected_AREAS={"FR","DE"}
Selected_TECHNOLOGIES={'Thermal', 'OldNuke', 'NewNuke', 'HydroRiver', 'HydroReservoir',
       'WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}

#### reading CSV files
TechParameters = pd.read_csv('CSV/input/Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=';',decimal=',',comment="#",skiprows=0)
areaConsumption = pd.read_csv('CSV/input/areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv('CSV/input/availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
ExchangeParameters = pd.read_csv('CSV/input/Hypothese_DE-FR_AREAS_AREAS.csv',sep=';',decimal=',',skiprows=0,comment="#")
#### Selection of subset
TechParameters=TechParameters[TechParameters.AREAS.isin(Selected_AREAS)&TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
areaConsumption=areaConsumption[areaConsumption.AREAS.isin(Selected_AREAS)]
availabilityFactor=availabilityFactor[availabilityFactor.AREAS.isin(Selected_AREAS)& availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

### small data cleaning
availabilityFactor.availabilityFactor[availabilityFactor.availabilityFactor>1]=1
model = GetElectricSystemModel_GestionMultiNode(areaConsumption,availabilityFactor,TechParameters,ExchangeParameters)
opt = SolverFactory('mosek')
results=opt.solve(model)
Variables=getVariables_panda(model)
Variables.keys()
Variables["energy"].head()
Variables["exchange"].head()
Variables["exchange"].exchange.sum()
Constraints= getConstraintsDual_panda(model)
Constraints.keys()

## 4. Storage operation <a class="anchor" id="storage"></a>
### 4.1. Optimisation of a storage market participation <a class="anchor" id="storageMarket"></a>
### 4.2. Simultaneous optimisation of storage and electric system <a class="anchor" id="storagecoupling"></a>


In [ ]:
Zones="FR"
year=2013

Selected_TECHNOLOGIES={'Thermal', 'OldNuke', 'WindOnShore',"Curtailement"}

#### reading CSV files
areaConsumption = pd.read_csv('CSV/input/areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv('CSV/input/availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv('CSV/input/Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

availabilityFactor.head()

availabilityFactor.pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='availabilityFactor').\
    iloc[1:240].plot(y="WindOnShore",kind="line").set_ylim(0,1)

pmax=10000
res= GetElectricSystemModel_GestionSingleNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,p_max=pmax,c_max=pmax*10)
areaConsumption = res["areaConsumption"]
model= res["model"]
stats=res["stats"]


PrixTotal
plt.figure(figsize=(12,5))
plt.xlabel('Number of requests every 10 minutes')

ax1 = areaConsumption.NewConsumption.plot(color='blue', grid=True, label='Count')
ax2 = areaConsumption.areaConsumption.plot(color='red', grid=True, secondary_y=True, label='Sum')

areaConsumption["NewConsumption"].max()
areaConsumption["Storage"].max()
plt.legend(h1+h2, l1+l2, loc=2)
plt.show()

plt.scatter(areaConsumption["NewConsumption"], areaConsumption["areaConsumption"])
plt.show() # Depending on whether you use IPython or interactive mode, etc.
areaConsumption.plot()
